In [1]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""Build the pure visual model"""
from mvcl.percept.metanet import MetaNet
from mvcl.model import MetaVisualLearner, config
from mvcl.custom import ObjectAffinityFeatures

#affinity = ObjectAffinityFeatures(3, 100)
metanet = MetaVisualLearner(None, config)

metanet.add_affinities(["texture", "color", "category"])

[Taichi] version 1.7.0, llvm 15.0.7, commit 7b58b0ff, osx, python 3.9.13


[I 04/09/24 07:30:31.230 2863736] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [5]:
B, C, W, H = (1, 3, 128, 128)
fake_ims = torch.randn([B,C,W,H])
targets = torch.randn([B,W,H,1])

#metanet.eval()
metanet.freeze_components()
metanet.calculate_object_affinity(fake_ims, targets)
#plt.imshow(fake_ims[0].permute(1,2,0))

torch.Size([1, 3, 16384, 58])
torch.Size([1, 16384, 58, 64]) torch.Size([1, 3, 64])


{'loss': 0.0, 'attn': None}

In [4]:
"""Write the code for the adaptive object affinity calculator"""
def calculate_object_affinity(self):
    """step 1: calculate the attention based on the component affinity key"""
    
    """step 2: calculate something real"""
    return 